In [1]:
import requests
import geopandas as gpd
import pandas as pd
import WGS1984

from tqdm import tqdm

In [2]:
def bus_compute(ox,oy,dx,dy,key):
    og = WGS1984.wgs84togcj02(ox,oy)
    gcj_origin = str(round(og[0],6)) + "," + str(round(og[1],6))
    de = WGS1984.wgs84togcj02(dx,dy)
    gcj_de = str(round(de[0],6)) + "," + str(round(de[1],6))

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"}
    URL = "https://restapi.amap.com/v3/direction/transit/integrated?parameters"
    keyword = {"origin": gcj_origin, "destination": gcj_de, "key": key, "time": "9:00","date": "2021-4-24","nightflag":"1","city":"深圳"}
    
    b = requests.get(URL, headers=headers, params=keyword)

    if b.status_code == 200:
        import json
        html = b.text
        json = json.loads(html)
        dis = int(json["route"]["transits"][0]["distance"])
        dur = int(json["route"]["transits"][0]["duration"])
    return dis,dur

In [3]:
work_point = gpd.read_file("data/point.geojson")
net = gpd.read_file("data/network.geojson")

In [23]:
keys = ["0327d3ae49ee9fc899fd12460c6e0fc2",
       "376e811e3c5fe9355aada73ad7473a36",
       "d8e6d4ab192c85ae0068064f5e7d1c06",
       "82ac160d4d62c7c851359ffaa2c4d2ce"]

a = 0

datas = []
for i in range(len(work_point)):
    ox = work_point.loc[i,"x"]
    oy = work_point.loc[i,"y"]
    oid = work_point.loc[i,"id"]

    a += 1
    if a <= 3 :
        key = keys[0]
    if 3 < a <= 6:
        key = keys[1]
    if 6 < a <= 9:
        key = keys[2] 
    
    for a in tqdm(range(len(net))):
        did = net.loc[a,"id"]
        dx = net.loc[a,"x"]
        dy = net.loc[a,"y"]

        try:
            da = bus_compute(ox,oy,dx,dy,key)
            data = {"oid":oid,
                    "did":did,
                    "distance":da[0],
                    "duration":round(da[1],3)}
            datas.append(data)
        except:
            pass
        
df = pd.DataFrame(columns=["oid","did","distance","duration"])  #构建一个空表
df = df.append(datas,ignore_index=True)
df.to_excel("/Users/creative/Desktop/cs数据.xlsx")       

  1%|          | 59/9201 [00:18<47:43,  3.19it/s]

In [9]:
df = pd.DataFrame(columns=["oid","did","distance","duration"])  #构建一个空表
df = df.append(datas,ignore_index=True)
df.to_excel("/Users/creative/Desktop/cs数据.xlsx") 

In [6]:
df

,oid,did,distance,duration
0,8,16,84845,12533
1,8,72,74775,13661
2,8,73,84128,11918
3,8,74,84516,12251
4,8,75,71351,12784
...,...,...,...,...
1785,8,3619,23109,9035
1786,8,3620,23555,9417
1787,8,3621,32301,8752
1788,8,3622,31094,7718
